In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = 'cache'

import pandas as pd
import numpy as np 

try:
    from tqdm.auto import tqdm
except ImportError:
    tqdm = lambda x: x
    
from txtai.embeddings import Embeddings
from txtai.pipeline import Similarity
# from txtai.pipeline import Tabular
# from txtai.workflow import Task
# from txtai.workflow import Workflow

from huggingface_hub import snapshot_download

In [2]:
filename = "ctgov_20230417.csv"
# ckptpath = "pritamdeka/S-PubMedBert-MS-MARCO"
# ckptpath = "microsoft/biogpt"
# ckptpath = "microsoft/BioGPT-Large-PubMedQA"
ckptpath = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
snapshot_download(repo_id=ckptpath,
                  repo_type="model",
                  cache_dir="cache")

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

'cache/models--sentence-transformers--multi-qa-mpnet-base-dot-v1/snapshots/69cf9082c6abd4f70bdf8fca0ca826b6b5d16ebc'

In [3]:
df = pd.read_csv(filename, index_col="nct_id")
df


,brief_title,official_title,baseline_measurements,brief_summaries,detailed_descriptions,criteria,gender,minimum_age,maximum_age,facilities,city,state,zip,country,recruitment_details,pre_assignment_details,study_type
nct_id,,,,,,,,,,,,,,,,,
NCT00000143,Studies of Ocular Complications of AIDS (SOCA)...,Studies of Ocular Complications of AIDS (SOCA)...,NaN,"To compare the newest CMV retinitis drug, cido...",Cytomegalovirus (CMV) is among the most freque...,Inclusion criteria:\n\nAge 13 years or older\n...,All,13 Years,NaN,"University of South Florida, MDC Box 21",Tampa,Texas,94143,United States,June 1997,NaN,Interventional
NCT00000378,Antidepressant Treatment of Melancholia in Lat...,Antidepressant Treatment of Melancholia in Lat...,NaN,The purpose of this study is to compare the sa...,To compare the efficacy and safety of a select...,Inclusion Criteria:\n\n-\n\nPatients must have...,All,60 Years,95 Years,1051 Riverside Drive,New York,New York,10032,United States,NaN,NaN,Interventional
NCT00000620,Action to Control Cardiovascular Risk in Diabe...,Action to Control Cardiovascular Risk in Diabe...,NaN,The purpose of this study is to prevent major ...,"BACKGROUND:\n\nCurrently, about 17 million Ame...",Inclusion Criteria:\n\nDiagnosed with type 2 d...,All,40 Years,79 Years,Wake Forest University,Winston-Salem,Washington,98195,United States,All participants had established type 2 diabet...,Eligible participants provided evidence of abi...,Interventional
NCT00001151,"Studies With 1,25-Dihydroxycholecalciferol","Studies With 1,25-Dihydroxycholecalciferol",NaN,Vitamin D in the diet undergoes changes in the...,"Patients with extreme resistance to 1,25-dihyd...",INCLUSION CRITERIA:\n\nPatients with hereditar...,All,NaN,NaN,"National Institutes of Health Clinical Center,...",Bethesda,Maryland,20892,United States,NaN,NaN,Interventional
NCT00001213,Cysteamine Eye Drops to Treat Corneal Crystals...,Trial of Topical Cysteamine in the Treatment o...,Although 328 participants were initially enrol...,Cystinosis is an inherited disease that result...,"Protocol 86-EI-0062 began as a randomized, dou...",INCLUSION CRITERIA:\n\nPatients must have a do...,All,2 Years,NaN,"National Institutes of Health Clinical Center,...",Bethesda,Maryland,20892,United States,NaN,NaN,Interventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCT05490771,Testing Copanlisib as a Potential Targeted Tre...,Phase II Study of Copanlisib in Patients With ...,NaN,This phase II MATCH treatment trial identifies...,PRIMARY OBJECTIVE:\n\nI. To evaluate the propo...,Inclusion Criteria:\n\nPatients must have met ...,All,18 Years,NaN,ECOG-ACRIN Cancer Research Group,Philadelphia,Pennsylvania,19103,United States,"Subprotocol Z1F was activated on June 20, 2018...",The PIK3CA mutations status was determined by ...,Interventional
NCT05502081,Clinical Study to Compare Efficacy and Safety ...,Clinical Study to Evaluate the Possible Effica...,0. Uninfected\n\nAmbulatory mild disease\n\nAs...,Introduction:\n\nCorona Virus induced disease ...,I. INTRODUCTION\n\n1.1. COVID-19 overview and ...,Inclusion Criteria:\n\nage more than 12 years ...,All,12 Years,NaN,El-gomhoria St,Mansoura,El-dkhalia,050,Egypt,from 1/11/2021 to 29/5/2022 at isolation hospi...,assignment is applied after admission of parti...,Interventional
NCT05594173,Chewing and Oral Processing of Solid Food,Chewing and Oral Processing of Solid Food in H...,NaN,Food texture modification is commonly used as ...,Aim: To explore chewing and oral processing be...,Inclusion Criteria:\n\nHealthy adults under ag...,All,18 Years,60 Years,Toronto Rehabilitation Institute - University ...,Toronto,Ontario,M5G 2A2,Canada,NaN,NaN,Observational


In [4]:
sem_fields = [
    "brief_title",
    "official_title",
    "brief_summaries",
    "detailed_descriptions", 
    "criteria"
]
df_sem = df[sem_fields]
df_sem


,brief_title,official_title,brief_summaries,detailed_descriptions,criteria
nct_id,,,,,
NCT00000143,Studies of Ocular Complications of AIDS (SOCA)...,Studies of Ocular Complications of AIDS (SOCA)...,"To compare the newest CMV retinitis drug, cido...",Cytomegalovirus (CMV) is among the most freque...,Inclusion criteria:\n\nAge 13 years or older\n...
NCT00000378,Antidepressant Treatment of Melancholia in Lat...,Antidepressant Treatment of Melancholia in Lat...,The purpose of this study is to compare the sa...,To compare the efficacy and safety of a select...,Inclusion Criteria:\n\n-\n\nPatients must have...
NCT00000620,Action to Control Cardiovascular Risk in Diabe...,Action to Control Cardiovascular Risk in Diabe...,The purpose of this study is to prevent major ...,"BACKGROUND:\n\nCurrently, about 17 million Ame...",Inclusion Criteria:\n\nDiagnosed with type 2 d...
NCT00001151,"Studies With 1,25-Dihydroxycholecalciferol","Studies With 1,25-Dihydroxycholecalciferol",Vitamin D in the diet undergoes changes in the...,"Patients with extreme resistance to 1,25-dihyd...",INCLUSION CRITERIA:\n\nPatients with hereditar...
NCT00001213,Cysteamine Eye Drops to Treat Corneal Crystals...,Trial of Topical Cysteamine in the Treatment o...,Cystinosis is an inherited disease that result...,"Protocol 86-EI-0062 began as a randomized, dou...",INCLUSION CRITERIA:\n\nPatients must have a do...
...,...,...,...,...,...
NCT05490771,Testing Copanlisib as a Potential Targeted Tre...,Phase II Study of Copanlisib in Patients With ...,This phase II MATCH treatment trial identifies...,PRIMARY OBJECTIVE:\n\nI. To evaluate the propo...,Inclusion Criteria:\n\nPatients must have met ...
NCT05502081,Clinical Study to Compare Efficacy and Safety ...,Clinical Study to Evaluate the Possible Effica...,Introduction:\n\nCorona Virus induced disease ...,I. INTRODUCTION\n\n1.1. COVID-19 overview and ...,Inclusion Criteria:\n\nage more than 12 years ...
NCT05594173,Chewing and Oral Processing of Solid Food,Chewing and Oral Processing of Solid Food in H...,Food texture modification is commonly used as ...,Aim: To explore chewing and oral processing be...,Inclusion Criteria:\n\nHealthy adults under ag...


In [6]:
# Concatenate all columns into a single "text" column
df_sem["text"] = df_sem.apply(
    lambda x: " ".join(str(x[field]) for field in df_sem.columns), axis=1
)

df_sem


/tmp/ipykernel_3112848/1840763510.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sem["text"] = df_sem.apply(


,brief_title,official_title,brief_summaries,detailed_descriptions,criteria,text
nct_id,,,,,,
NCT00000143,Studies of Ocular Complications of AIDS (SOCA)...,Studies of Ocular Complications of AIDS (SOCA)...,"To compare the newest CMV retinitis drug, cido...",Cytomegalovirus (CMV) is among the most freque...,Inclusion criteria:\n\nAge 13 years or older\n...,Studies of Ocular Complications of AIDS (SOCA)...
NCT00000378,Antidepressant Treatment of Melancholia in Lat...,Antidepressant Treatment of Melancholia in Lat...,The purpose of this study is to compare the sa...,To compare the efficacy and safety of a select...,Inclusion Criteria:\n\n-\n\nPatients must have...,Antidepressant Treatment of Melancholia in Lat...
NCT00000620,Action to Control Cardiovascular Risk in Diabe...,Action to Control Cardiovascular Risk in Diabe...,The purpose of this study is to prevent major ...,"BACKGROUND:\n\nCurrently, about 17 million Ame...",Inclusion Criteria:\n\nDiagnosed with type 2 d...,Action to Control Cardiovascular Risk in Diabe...
NCT00001151,"Studies With 1,25-Dihydroxycholecalciferol","Studies With 1,25-Dihydroxycholecalciferol",Vitamin D in the diet undergoes changes in the...,"Patients with extreme resistance to 1,25-dihyd...",INCLUSION CRITERIA:\n\nPatients with hereditar...,"Studies With 1,25-Dihydroxycholecalciferol Stu..."
NCT00001213,Cysteamine Eye Drops to Treat Corneal Crystals...,Trial of Topical Cysteamine in the Treatment o...,Cystinosis is an inherited disease that result...,"Protocol 86-EI-0062 began as a randomized, dou...",INCLUSION CRITERIA:\n\nPatients must have a do...,Cysteamine Eye Drops to Treat Corneal Crystals...
...,...,...,...,...,...,...
NCT05490771,Testing Copanlisib as a Potential Targeted Tre...,Phase II Study of Copanlisib in Patients With ...,This phase II MATCH treatment trial identifies...,PRIMARY OBJECTIVE:\n\nI. To evaluate the propo...,Inclusion Criteria:\n\nPatients must have met ...,Testing Copanlisib as a Potential Targeted Tre...
NCT05502081,Clinical Study to Compare Efficacy and Safety ...,Clinical Study to Evaluate the Possible Effica...,Introduction:\n\nCorona Virus induced disease ...,I. INTRODUCTION\n\n1.1. COVID-19 overview and ...,Inclusion Criteria:\n\nage more than 12 years ...,Clinical Study to Compare Efficacy and Safety ...
NCT05594173,Chewing and Oral Processing of Solid Food,Chewing and Oral Processing of Solid Food in H...,Food texture modification is commonly used as ...,Aim: To explore chewing and oral processing be...,Inclusion Criteria:\n\nHealthy adults under ag...,Chewing and Oral Processing of Solid Food Chew...


# Semantic Search

In [7]:
def stream(dataframe, field, limit):
    for index, row in tqdm(dataframe.iloc[:limit].iterrows()):
        yield (index, row[field], None)


def search(query):
    return [(result["score"], result["text"]) for result in embeddings.search(query, limit=50)]


def ranksearch(query):
    results = [text for _, text in search(query)]
    return [(score, results[x]) for x, score in similarity(query, results)]


In [8]:
embeddings = Embeddings({
    "path": ckptpath, 
    "content": True,
})

# Create similarity instance for re-ranking
similarity = Similarity(ckptpath)


# Create embeddings model, backed by sentence-transformers & transformers, enable content storage
embeddings.index(stream(df_sem, "text", 40000))


Some weights of the model checkpoint at sentence-transformers/multi-qa-mpnet-base-dot-v1 were not used when initializing MPNetForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/multi-qa-mpnet-base-dot-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRA

0it [00:00, ?it/s]

In [ ]:
queries = [
    "diabetes", 
    "hypertension",
]

for query in queries:
    display(query, ranksearch(query)[:2])

# Tabular data

In [12]:
from txtai.pipeline import Tabular
from txtai.workflow import Task, Workflow

# Create tabular instance mapping input.csv fields
tabular = Tabular("nct_id", sem_fields)

# Create workflow
workflow = Workflow([Task(tabular)])

# Print 5 rows of input.csv via workflow
list(workflow([filename]))[:5]


[('NCT00000143',
  'Studies of Ocular Complications of AIDS (SOCA)--Ganciclovir-Cidofovir CMV Retinitis Trial (GCCRT). Studies of Ocular Complications of AIDS (SOCA)--Ganciclovir-Cidofovir CMV Retinitis Trial (GCCRT). To compare the newest CMV retinitis drug, cidofovir, with a regimen of the ganciclovir intraocular device plus oral ganciclovir with respect to efficacy in preventing vision loss.\n\nTo compare a treatment regimen that incorporates highly active local therapy (ganciclovir device) with a treatment regimen that does not.. Cytomegalovirus (CMV) is among the most frequently encountered opportunistic infections in patients with AIDS. In the era of prophylaxis for pneumocystic pneumonia, CMV disease is estimated to affect 45 percent of patients with AIDS sometime between the diagnosis of AIDS and death. Retinitis has been estimated to account for up to 85 percent of CMV disease in these patients, making CMV retinitis the most common ocular infection encountered. CMV retinitis i

In [13]:
from txtai.embeddings import Embeddings

# Embeddings with sentence-transformers backend
embeddings = Embeddings({
    "path": ckptpath
    })

# Index subset of CORD-19 data
data = list(workflow([filename]))
embeddings.index(data)


The Health Influences of Puberty (HIP) Study. Combined Influence of Puberty and Obesity on Insulin Resistance in Adolescents. The Health Influences of Puberty (HIP) Study is designed to explore the relationships between puberty and the onset of type 2 diabetes in adolescents. The results of this study will help us better understand how to prevent type 2 diabetes in these youth. Children go through many changes during puberty, including important hormonal and behavioral alterations. Among these changes, it has long been known that, during puberty, insulin does not work as well as it does before and after puberty. This is called physiologic insulin resistance. In healthy children, this does not cause diabetes or affect blood sugar in any way because the body is able to compensate by making more insulin. Indeed, this is thought to be an important part of the adolescent growth spurt. However, in some children with increased risk for developing type 2 diabetes due to obesity and genetics, t

In [14]:

for uid, _ in embeddings.search("diabetes"):
    title = [text for idx, text, _ in data if idx == uid][0]
    print(title, uid)

The Health Influences of Puberty (HIP) Study. Combined Influence of Puberty and Obesity on Insulin Resistance in Adolescents. The Health Influences of Puberty (HIP) Study is designed to explore the relationships between puberty and the onset of type 2 diabetes in adolescents. The results of this study will help us better understand how to prevent type 2 diabetes in these youth. Children go through many changes during puberty, including important hormonal and behavioral alterations. Among these changes, it has long been known that, during puberty, insulin does not work as well as it does before and after puberty. This is called physiologic insulin resistance. In healthy children, this does not cause diabetes or affect blood sugar in any way because the body is able to compensate by making more insulin. Indeed, this is thought to be an important part of the adolescent growth spurt. However, in some children with increased risk for developing type 2 diabetes due to obesity and genetics, t

In [15]:
from txtai.embeddings import Embeddings
import json 

# Embeddings with sentence-transformers backend
embeddings = Embeddings({
    "path": ckptpath, 
    "content": True
})

# Index subset of CORD-19 data
data = list(workflow([filename]))
embeddings.index(data)


In [18]:
query = f'select {",".join([column for columns in df_sem.colums])} from txtai where similar("diabetes")'
for uid, _ in embeddings.search(query):
    title = [text for idx, text, _ in data if idx == uid][0]
    print(title, uid)


SyntaxError: invalid syntax (3838325138.py, line 1)